# **추출 체인(Extraction Chain) 구축하기**

이 노트북에서는 **채팅 모델(Chat Models)**의 **도구 호출(Tool Calling)** 기능을 사용하여 **비정형 텍스트에서 구조화된 정보를 추출**하는 방법을 다룹니다. 또한 이 맥락에서 **Few-Shot 프롬프팅(Few-Shot Prompting)**을 사용하여 성능을 개선하는 방법을 시연할 것입니다.

## **스키마 (The Schema)**  

먼저, 텍스트에서 어떤 정보를 추출할 것인지 **정의**해야 합니다.  

이를 위해 **Pydantic**을 사용하여 **개인 정보(personal information)**를 추출하기 위한 예제 **스키마(schema)**를 정의할 것입니다.

In [2]:
# !pip install -qU \
# python-dotenv \
# langchain \
# langchain-community \
# openai \
# anthropic \
# langchain-openai \
# langchain-anthropic \
# langchain-google-genai \
# python-dotenv

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from typing import Optional  
from pydantic import BaseModel, Field 

class Person(BaseModel):
    """
    사람에 대한 정보.   
    """
    # 이 Doc-string은 LLM에 'Person' 스키마에 대한 설명으로 전달됩니다.
    # 사람의 이름
    name: Optional[str] = Field(default=None, description="사람의 이름")
    # 사람의 머리 색상
    hair_color: Optional[str] = Field(default=None, description="사람의 머리 색상")
    # 사람의 키 (미터 단위)
    height_in_meters: Optional[str] = Field(
        default=None, 
        description="미터 단위로 측정된 키"
    )

## **스키마 정의의 두 가지 사례**

1. **속성(attributes)**과 **스키마(schema)** 를 Pydantic으로 문서화:  
   - 이 정보는 LLM에 전달되며, Pydantic 스키마를 통해 명확하게 정의되어 정보 추출의 품질을 개선하는 데 사용됩니다.
     
<pr></pr>

2. **LLM이 정보를 지어내지 않도록 하세요!**  
   - 위에서 각 속성에 `Optional`을 사용하여 LLM이 답을 모를 경우 `None`을 반환할 수 있도록 했습니다.
  
최상의 성능을 얻으려면 **스키마를 잘 문서화**하고, 텍스트에 추출할 정보가 없을 경우 모델이 결과를 **강제로 반환하지 않도록** 설정합니다.  


## **추출기 (The Extractor)**

이제 위에서 정의한 **스키마(schema)**를 사용하여 **정보 추출기(Information Extractor)**를 만들어 봅니다.

In [4]:
from typing import Optional   
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field  

# 사용자 정의 프롬프트 템플릿 정의
# 텍스트에서 정보를 추출하기 위한 명확한 지침과 추가 컨텍스트를 제공합니다.

prompt_template = ChatPromptTemplate(
    [
        (
            "system",
            "당신은 전문 정보 추출 알고리즘입니다. "
            "텍스트에서 관련 정보만 추출하세요. "
            "추출해야 할 속성의 값을 알지 못할 경우, "
            "해당 속성의 값으로 null을 반환하세요.",
        ),
        ("human", "{text}"),   # 사용자 입력 텍스트를 프롬프트에 전달
    ]
)

**기능/도구 호출(Function/Tool Calling)**을 지원하는 모델을 사용해야 합니다.

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [6]:
# LLM에서 구조화된 출력을 생성하도록 스키마 바인딩
structured_llm = llm.with_structured_output(schema=Person)
structured_llm

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000152841606D0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001528416E880>, root_client=<openai.OpenAI object at 0x0000015284031400>, root_async_client=<openai.AsyncOpenAI object at 0x0000015284160730>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Person', 'description': '사람에 대한 정보.   ', 'parameters': {'properties': {'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': '사람의 이름'}, 'hair_color': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': '사람의 머리 색상'}, 'height_in_meters': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'description': '미터 단위로 측정된 키'}}, 'type': 'object'}}}], 'parallel_tool_calls': False, 'tool_choice': {'type': 'function', 'function': {'name': 'Pe

In [9]:
text = "도날트 트럼프 대통령의 키는 186센티이고 금발입니다. 그의 친한 친구 John Doe는 은발입니다. 그 사람의 고향은 어디인가요?"

prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='도날트 트럼프', hair_color='금발', height_in_meters='1.86')

LLM은 생성 모델이므로, 센티미터로 제공된 신장의 정보를 미터로 정확하게 추출하는 등의 놀라운 작업을 수행할 수 있습니다! 또한 스키마에 정의된 이름, 머리색, 키 외의 다른 내용은 무시하고 답변을 생성합니다.


## 다중 엔터티

많은 경우, 단일 엔티티가 아닌 여러 엔티티를 추출해야 합니다. 이는 Pydantic에서 모델을 서로 중첩하여 쉽게 구현할 수 있습니다.

In [11]:
from typing import List

class Data(BaseModel):
    """
    여러 사람들에 대한 추출된 데이터.
    """
    peoples: List[Person]   # 여러 사람의 정보를 추출하기 위해 'Person' 모델의 리스트를 정의

In [12]:
structured_llm = llm.with_structured_output(schema=Data)

text = "제 이름은 제프이고, 제 머리는 검은색이고 키는 6피트입니다. 안나는 저와 같은 색의 머리를 가지고 있습니다."

prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Data(peoples=[Person(name='제프', hair_color='검은색', height_in_meters='1.83'), Person(name='안나', hair_color='검은색', height_in_meters=None)])

**여러 엔티티**를 추출할 수 있도록 스키마가 설계되면, 텍스트에 관련 정보가 없을 경우 **빈 리스트(empty list)**를 반환하여 **아무런 엔티티도 추출하지 않을 수 있습니다.**  

이는 일반적으로 **좋은 설계**입니다! 이를 통해 모델이 해당 엔티티를 반드시 감지하도록 강제하지 않을 수 있습니다.  

## **Few-Shot Prompting**  

LLM 애플리케이션의 동작은 **Few-Shot 프롬프팅**을 사용하여 조정할 수 있습니다.  

**챗 모델(Chat Models)**의 경우, 원하는 동작을 보여주는 **입력(input)**과 **응답(response)** 메시지 쌍의 시퀀스로 구성될 수 있습니다.  

예를 들어, `user`와 `assistant` **메시지**가 번갈아 가며 나타나는 구조를 통해 `🦜` 기호의 의미를 전달할 수 있습니다.

In [13]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7


**구조화된 출력(Structured Output)**은 종종 도구 호출(Tool Calling)을 기반으로 작동합니다. 이는 일반적으로 **도구 호출을 포함하는 AI 메시지(AI messages)**와 **도구 호출의 결과를 포함하는 도구 메시지(Tool messages)**의 생성을 포함합니다.  
LangChain의 `tool_example_to_messages` 함수는 각 업체의 LLM에 맞추어 도구 호출 형식을 자동으로 포맷합니다. 

---

### **LangChain의 유틸리티 함수: `tool_example_to_messages`**  
각 모델 제공업체(OpenAI, Anthropic 등)마다 도구 호출 형식이 조금씩 다릅니다. LangChain의 tool_example_to_messages 함수는 Pydantic 객체만 있으면, 이를 모델 제공업체의 요구사항에 맞는 형식으로 자동 변환해주는 도구입니다. tool_example_to_messages는 개발자가 복잡한 포맷을 신경 쓰지 않고, Pydantic 객체만 제공하면 자동으로 올바른 메시지 형식을 만들어줍니다.

---

###  **동작 순서**  

1. 입력 문자열: 사용자의 요청이나 질문을 문자열로 제공합니다.  
2. Pydantic 객체: 사용할 도구나 함수의 입력 형식을 Pydantic 모델로 정의합니다.  
3. 자동 포맷: LangChain이 이 두 가지를 조합해 모델 제공업체(OpenAI, Anthropic 등)에 맞는 메시지 포맷으로 자동 변환합니다.


In [17]:
# 도구 호출 예제를 메시지로 변환하기 위한 유틸리티 함수
from langchain_core.utils.function_calling import tool_example_to_messages

# Few-Shot Prompting 예제 데이터 정의
# 입력 텍스트와 모델이 추출해야 할 데이터의 목표 형식(Target Schema) 쌍으로 정의
examples = [
    (
        "바다는 넓고 푸르다. 깊이가 20,000피트가 넘습니다.",
        Data(peoples=[]),   # 모델이 추출할 데이터 - 사람 정보가 없는 경우
    ),
    (
        "차박차박 유튜버는 프랑스에서 스페인까지 먼 여행을 했습니다.",
        Data(peoples=[Person(name="차박차박", height_in_meters=None, hair_color=None)]),  # 모델이 추출할 데이터 - 차박차박에 대한 정보 추출
    ),
]

# 메시지 목록 초기화
messages = []

# 예제 데이터를 메시지로 변환
for txt, expected_output in examples:
    if expected_output.peoples:  # 사람 정보가 있는 경우
        response = "Detected people."
    else:
        response = "Detected no people."

    # tool_example_to_messages 함수를 사용해 입력 텍스트와 도구 호출 데이터를 메시지로 변환
    messages.extend(tool_example_to_messages(txt, [expected_output], ai_response=response))

결과를 살펴보면 다음 두 쌍의 예시가 8개의 메시지를 생성한 것을 알 수 있습니다.

In [18]:
messages

[HumanMessage(content='바다는 넓고 푸르다. 깊이가 20,000피트가 넘습니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'a5e1d83c-16ad-4983-9009-a3cf5937ccca', 'type': 'function', 'function': {'name': 'Data', 'arguments': '{"peoples":[]}'}}]}, response_metadata={}, tool_calls=[{'name': 'Data', 'args': {'peoples': []}, 'id': 'a5e1d83c-16ad-4983-9009-a3cf5937ccca', 'type': 'tool_call'}]),
 ToolMessage(content='You have correctly called this tool.', tool_call_id='a5e1d83c-16ad-4983-9009-a3cf5937ccca'),
 AIMessage(content='Detected no people.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='차박차박 유튜버는 프랑스에서 스페인까지 먼 여행을 했습니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '89b74e2f-0677-4a5d-9ba7-90f9ba9e1e7b', 'type': 'function', 'function': {'name': 'Data', 'arguments': '{"peoples":[{"name":"차박차박","hair_color":null,"height_in_meters":null}]}'}}]}, response_meta

In [20]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

바다는 넓고 푸르다. 깊이가 20,000피트가 넘습니다.
================================== Ai Message ==================================
Tool Calls:
  Data (a5e1d83c-16ad-4983-9009-a3cf5937ccca)
 Call ID: a5e1d83c-16ad-4983-9009-a3cf5937ccca
  Args:
    peoples: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

Detected no people.
================================ Human Message =================================

차박차박 유튜버는 프랑스에서 스페인까지 먼 여행을 했습니다.
================================== Ai Message ==================================
Tool Calls:
  Data (89b74e2f-0677-4a5d-9ba7-90f9ba9e1e7b)
 Call ID: 89b74e2f-0677-4a5d-9ba7-90f9ba9e1e7b
  Args:
    peoples: [{'name': '차박차박', 'hair_color': None, 'height_in_meters': None}]
================================= Tool Message ====================

----
이 메시지들을 Few-Shot Example로 Prompt에 포함할와 포함하지 않을 때의 성능을 비교해봅니다.   
다음은, 사람이 포함되지 않은 메시지에 대한 llm의 response 비교 입니다.

In [27]:
message_no_people = {
    "role": "user",
    "content": "태양계는 크지만, 지구에는 달이 단 하나뿐입니다.",
}

structured_response = llm.with_structured_output(schema=Data)
answer = structured_response.invoke([message_no_people])
answer

Data(peoples=[Person(name='태양계', hair_color='없음', height_in_meters='0'), Person(name='지구', hair_color='푸른색', height_in_meters='0'), Person(name='달', hair_color='회색', height_in_meters='0')])

In [28]:
print(answer.peoples)

[Person(name='태양계', hair_color='없음', height_in_meters='0'), Person(name='지구', hair_color='푸른색', height_in_meters='0'), Person(name='달', hair_color='회색', height_in_meters='0')]


이 예제에서 모델은 사람에 대한 잘못된 응답을 생성할 가능성이 있습니다.  

그러나 "Detected no people" 사례가 포함되어 있는 **Few-Shot Example**을 제공할 경우, 모델이 올바르게 작동하도록 유도할 수 있습니다.

In [30]:
answer = structured_response.invoke(messages + [message_no_people])
answer

Data(peoples=[])